### Data engineering exploration: League of Legends edition

You might be wondering, where is the point of this project? The response is straightforward: I have been a huge fan of League of Legends since my high school days, so I thought that using data of this game could be great to add to my portfolio in data engineering. 

Despite there are so much data about this game scattered across the internet, I considered that will be fun to create my own dataset with the most relevant information of my recent games using the Riot API, tool that allow us to collect many data of every game of the company. 

We will need three libraries in this project:

* Requests: an elegant and simple HTTP library for Python that we will use to create the API calls.

* Pandas: one of the most famous Python libraries for data analysis and manipulation.

* Re: a library to work with regular expressions in Python.

Additionally, we import the API key alocated if 'creds' file, which enables us to make calls and extract data. Since it is linked to my account, it's crucial not to disclose it.

In [1]:
import requests
import pandas as pd
from creds import api_key
import re

### Part 1: SUMMONER-V4 Endpoint

In this part we will utilize  the SUMMONER-V4 endpoint to extract account data. My summoner name, or nickname, is 'Masgt7', so that is the account we will use in this demonstration.

In [2]:
url_summoner = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Masgt7' + '?api_key=' + api_key

r = requests.get (url_summoner)
r.status_code

200

We obtained 200 response code, which means that the call was succesfull. To see the data, we must use the json method

In [3]:
player_info = r.json()
player_info

{'id': '7xKp5xDyqk1u0_RKrh8SdaSHYhjpThiq7XXJscT2DQx273I',
 'accountId': 'NcGfPSobpg1gNOsZBjrQNCVyeIg7MyijVoglw0WBQioOST0',
 'puuid': 'uFIekhnHnZjaIXG_IG5y2Kb_UbBsEv57LksBUe6qMh0h0MPBKg8xoWtsbQkvXcPhlHLNGn8xZwqzGQ',
 'name': 'Masgt7',
 'profileIconId': 940,
 'revisionDate': 1701025857377,
 'summonerLevel': 74}

There is some information about my accout. In order to obtain the data about my recent games, we have to use the Platform Unique Identifier ('puuid') 

In [4]:
puuid = player_info['puuid']

### Part 2: MATCH-V5 Endpoint
Similar to the previous call, we will now extract the identifiers of my recent games. In the new URL for extracting this data, we've included a new variable, 'num_games,' which represents the number of games to retrieve. It can range from 0 to 100, but for this demonstration, we will select 20

In [5]:
num_games = 20

url_match_history = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?type=normal&start=0&count=' + str(num_games) + '&api_key=' + api_key
r = requests.get(url_match_history)

if r.status_code == 200:
    game_id = r.json()
    print (game_id)
else:
    print ('Error', r.status_code, ':',) 

['EUW1_6693543692', 'EUW1_6693492561', 'EUW1_6693435700', 'EUW1_6693383878', 'EUW1_6691848158', 'EUW1_6688283685', 'EUW1_6688256964', 'EUW1_6688236897', 'EUW1_6688218767', 'EUW1_6686773929', 'EUW1_6686744190', 'EUW1_6684157765', 'EUW1_6684133215', 'EUW1_6682813114', 'EUW1_6682767095', 'EUW1_6682519608', 'EUW1_6678781224', 'EUW1_6678748896', 'EUW1_6678696426', 'EUW1_6678677984']


With the identifiers of the games, we can now generate a list that contains dataframes created from the data of each match.

In [6]:
list_df = []

for index, identifier in enumerate(game_id):
    url_game_info = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + identifier + '?api_key=' + api_key
    game_info = requests.get(url_game_info)
    index_player = game_info.json()['metadata']['participants'].index(puuid)
    game_data = game_info.json()['info']['participants'][index_player]
    game_df = pd.json_normalize(game_data)
    list_df.append(pd.DataFrame(game_df))

In [7]:
for i in range(len(game_id)):
    print (list_df[i].shape)

(1, 263)
(1, 262)
(1, 263)
(1, 264)
(1, 264)
(1, 261)
(1, 263)
(1, 263)
(1, 263)
(1, 263)
(1, 262)
(1, 267)
(1, 268)
(1, 267)
(1, 268)
(1, 268)
(1, 267)
(1, 267)
(1, 266)
(1, 266)


As we can see, this dataframes don't have the same columns, so we can't consolidate them in one. To achieve this, we will drop any columns that do not appear in every dataframe

In [8]:
common_columns = list(set.intersection(*(set(df.columns) for df in list_df)))

game_history = pd.concat(list_df)
game_history = game_history [common_columns]

game_history.shape

(20, 260)

The result is a dataframe with 260 columns. Let's take a look at the first five rows in the next cell.

In [9]:
game_history.head()

,champLevel,teamPosition,enemyMissingPings,challenges.takedownOnFirstTurret,totalEnemyJungleMinionsKilled,challenges.skillshotsHit,missions.playerScore2,challenges.elderDragonKillsWithOpposingSoul,challenges.gameLength,totalHealsOnTeammates,...,challenges.bountyGold,firstTowerAssist,firstBloodAssist,neutralMinionsKilled,totalDamageTaken,challenges.landSkillShotsEarlyGame,playerScore6,challenges.killAfterHiddenWithAlly,commandPings,challenges.multiKillOneSpell
0,16,,0,1,0,20,0,0,936.916152,0,...,450,False,False,0,17429,9,0,8,0,0
0,18,,0,0,0,67,0,0,1488.946525,0,...,450,False,False,0,31887,21,0,6,1,0
0,18,,0,1,0,26,0,0,1577.065913,0,...,500,False,True,0,39686,8,0,17,0,0
0,18,,0,0,0,119,0,0,1258.428256,0,...,600,True,False,0,20797,30,0,3,1,0
0,16,,1,0,0,0,0,0,913.553046,0,...,150,True,False,0,12940,0,0,14,0,0


For the sake of clarity, we will remove the text preceding the dot in columns where it occurs . We will do this using regular expressions

In [10]:
def remove_dot (string):
    pattern = r'.*?\.(.*)'
    result = re.match(pattern, string)
    if result:
        return result.group(1)
    else:
        return string

In [11]:
df_columns = list(game_history.columns)

for i,j in enumerate(df_columns):
    df_columns[i] = remove_dot(j)

game_history.columns = df_columns

game_history.head(1)

,champLevel,teamPosition,enemyMissingPings,takedownOnFirstTurret,totalEnemyJungleMinionsKilled,skillshotsHit,playerScore2,elderDragonKillsWithOpposingSoul,gameLength,totalHealsOnTeammates,...,bountyGold,firstTowerAssist,firstBloodAssist,neutralMinionsKilled,totalDamageTaken,landSkillShotsEarlyGame,playerScore6,killAfterHiddenWithAlly,commandPings,multiKillOneSpell
0,16,,0,1,0,20,0,0,936.916152,0,...,450,False,False,0,17429,9,0,8,0,0


The API provides detailed information about each game, but for clarity, we will select only a subset of columns with the most relevant ones

In [12]:
game_history = game_history [['kills', 'deaths', 'assists', 'kda', 'championName', 'individualPosition', 'killParticipation', 'summoner1Casts', 'summoner2Casts', 
                       'largestMultiKill', 'totalDamageDealtToChampions', 'teamDamagePercentage', 'totalDamageTaken', 'damageTakenOnTeamPercentage',
                       'largestKillingSpree', 'allInPings', 'assistMePings', 'baitPings', 
                       'basicPings', 'commandPings', 'dangerPings', 'enemyVisionPings', 'getBackPings', 'visionClearedPings', 'holdPings', 'needVisionPings', 
                       'onMyWayPings', 'gameLength', 'goldEarned', 'goldPerMinute', 'wardsPlaced', 'wardsKilled', 'win']]

In [13]:
game_history.shape

(20, 33)

The result of this selection is that we have passed from an enormous dataframe with high dimensionality to a smaller one with 33 columns that describe my performance in the games in a concise way

Now, we will change the index of each row in order to see directly the game that it describes.

In [14]:
game_history.reset_index(drop=True, inplace=True)
game_history.index += 1

game_history.head()

,kills,deaths,assists,kda,championName,individualPosition,killParticipation,summoner1Casts,summoner2Casts,largestMultiKill,...,visionClearedPings,holdPings,needVisionPings,onMyWayPings,gameLength,goldEarned,goldPerMinute,wardsPlaced,wardsKilled,win
1,12,9,22,3.777778,LeeSin,Invalid,0.618182,3,15,2,...,0,0,0,0,936.916152,12214,782.240657,0,0,True
2,11,11,26,3.363636,Xayah,Invalid,0.569231,4,3,1,...,0,0,0,0,1488.946525,17781,716.549853,0,0,False
3,8,15,36,2.933333,Morgana,Invalid,0.687500,4,17,2,...,0,0,0,0,1577.065913,16489,627.348839,0,0,False
4,12,6,30,7.000000,Corki,Invalid,0.857143,5,8,3,...,0,0,0,0,1258.428256,16559,789.513684,0,0,True
5,23,5,16,7.800000,Tristana,Invalid,0.750000,2,3,4,...,0,0,0,0,913.553046,14571,957.009865,0,0,True


Finally, we will do some transformations in some variables to provide a clearer view of them.

We multiply the percentages by 100 and round them

In [15]:
percentages = ['killParticipation','teamDamagePercentage', 'damageTakenOnTeamPercentage']

for i in percentages:
        game_history[i] = round (game_history[i] * 100, 2)

Also, aiming to reduce even more the dimensionality we can sum all the pings in just one column. Pings in LoL are important because they are an efficient way to comunicate your team several events in the game, but in our example we do not need data of them with this detail

In [16]:
pings = game_history.loc[:, 'allInPings':'onMyWayPings']
totalPings = pings.sum(axis=1)

game_history.insert(15, 'totalPings', totalPings)

game_history = game_history.drop(pings, axis=1)

Lastly, we will transform the column containing the duration of the game from seconds to minutes and round two columns with float values

In [18]:
game_history['gameLength'] = pd.to_datetime(game_history['gameLength'], unit = 'm').dt.strftime('%H:%M')

game_history['goldPerMinute'] = round (game_history['goldPerMinute'], 2)

game_history['kda'] = round (game_history['kda'], 2)

Finally, here is the resulting dataset that we saved as a CSV file in the project root.

In [28]:
pd.set_option('display.max_columns', None)
display(game_history)

,kills,deaths,assists,kda,championName,individualPosition,killParticipation,summoner1Casts,summoner2Casts,largestMultiKill,totalDamageDealtToChampions,teamDamagePercentage,totalDamageTaken,damageTakenOnTeamPercentage,largestKillingSpree,totalPings,gameLength,goldEarned,goldPerMinute,wardsPlaced,wardsKilled,win
1,12,9,22,3.78,LeeSin,Invalid,61.82,3,15,2,20499,20.19,17429,20.71,2,0,15:36,12214,782.24,0,0,True
2,11,11,26,3.36,Xayah,Invalid,56.92,4,3,1,45642,22.92,31887,18.68,3,1,00:48,17781,716.55,0,0,False
3,8,15,36,2.93,Morgana,Invalid,68.75,4,17,2,27141,14.50,39686,15.33,2,0,02:17,16489,627.35,0,0,False
4,12,6,30,7.00,Corki,Invalid,85.71,5,8,3,60120,30.14,20797,13.11,4,1,20:58,16559,789.51,0,0,True
5,23,5,16,7.80,Tristana,Invalid,75.00,2,3,4,23585,22.87,12940,15.59,10,0,15:13,14571,957.01,0,0,True
6,6,10,28,3.40,Shen,Invalid,77.27,5,33,1,25707,15.52,58100,32.18,2,0,23:39,14124,596.83,0,0,False
7,15,13,24,3.00,Xayah,Invalid,60.94,5,6,2,27617,22.45,30754,16.14,3,1,19:24,15387,792.89,0,0,False
8,19,10,9,2.80,Jax,Invalid,54.90,6,36,2,26658,24.52,35250,22.04,3,7,23:08,17809,769.82,0,0,True
9,6,10,14,2.00,Camille,Invalid,50.00,4,16,2,13649,15.61,24784,12.59,2,0,16:32,12063,729.31,0,0,True
10,8,6,28,6.00,Corki,Invalid,72.00,4,5,2,36196,28.00,22136,19.16,4,1,21:00,14624,696.34,0,0,True


In [20]:
game_history.to_csv('game_history.csv', mode = 'x', sep=',')

That's all. Thank you for reading and checking my project!